# Chapter 17: Causal Survival Analysis

So far we have been concerned with causal questions about the treatment effects on outcomes occuring at at a particular time point.

However, many causal questions are concerned with treatment effects on the time until the occurrence of an event of interest. For example, we may want to estimate the causal effect of smoking cessation on the time until death, whenever death occurs. This is an example of a *survival analysis*.

This chapter outlines basic techniques for survival analysis in the simplified setting of time-fixed treatments.

## 17.1 Hazards and risks

Suppose we want to estimate the avg. causal effect of smoking cessation $A$ (1: yes, 0: no) on the time to death $T$ with time measured from the start of follow-up. This is an example of survival analysis: the outcome is time to an event of interest that can occur at any time after the start of follow-up.

In most followup-studies, the event of interest is not observed to happen for all, or even the majority of individuals in the study. This is because most follow-up studies have a date after which there is no information on any individuals: the *administrative end of follow-up*. We know that they survived beyond the administrative end of follow-up, but we do not know for how much longer.

*Administrative censoring* is a problem intrinsic to survival analysis.

To give an example, let's say that the month of death $T$ can take values from 1 to 120. $T$ is known for 102 treated ($A=1$) and 216 untreated ($A=0$) individuals who died during the follow-up, and is administratively censored. Therefore we cannot compute the mean survival $\hat E[T]$ as we did previously. Rather, in survival analysis we need to use other measures that can accomodate administrative censoring.

Some common measures are the survival probability, the the risk, and the hazard.

The *survival probability* ($\Pr[T>k]$), or simply the survival at month $k$, is the proportion of individuals who survived through time $k$. The curve starts at $\Pr[T>0]=1$ for $k=0$ and then decreases monotonically with subsequent values of $k$.

The *risk*, or cumulative incidence at time $k$ is $1-\Pr[T>k] = \Pr[T\leq k]$. The cumulative incidence curve starts at $\Pr[T\leq 0]=0$ and increases monotonically during the follow-up.

We can also calculate at time $k$ the proportion of individuals who develop the event among those who had not developed it before $k$. This is the *hazard*, defined as $\Pr[T=k|T>k-1]$.

A frequent approach to quantify the treatment effect in survival analyses is to estimate the ratio of the hazards in the treated and the untreated, known as the *hazard ratio*. However, because the hazard ratio is problematic, this book focuses on survival/risk over hazard.



## 17.2 From hazards to risks

Typically with survival analyses we will use the long format to store our data. This means that each row corresponds to a person-time. To encode survival information through $k$ in the person-time data format, we use a time-varying indicator of event $D_k$. For each person at month $k$, the indicator $D_k$ takes value 1 if $T\leq k$ and value 0 if $T>k$. Meaning, it is 0 until they hit "death".

In a person-time data format, for an individual at time $k$, it includes the indicator $D_{k+1}$.

Using time-varying outcome variable $D_k$, we can define survival $k$ as $\Pr[D_k=0]$, which is equal to $\Pr[T>k]$ and risk at $k$ as $\Pr[D_k=1]$ which is equal to $\Pr[T\leq k]$. The hazard at $k$ is defined as $\Pr[D_k=1|D_{k-1}=0]$. For $k=1$ the hazard is equal to the risk because everybody is by definition alive at $k=0$.

The survival probability at $k$ is the product of the conditional probabilities of having survived each interval between $0$ and $k$. For example, the survival at $k=2$, $\Pr[D_2=0]$ is equal to the survival at $k=1$,$\Pr[D_1=0]$ times the survival probability at $k=2$ conditional on having survived through $k=1$, $\Pr[D_2=0|D_1=0]$.

More generally, the survival at $k$ is:

$$\Pr[D_k=0]=\prod_{m=1}^{k}\Pr[D_m=0|D_{m-1}=0]$$

In other words, the survival at $k$ equals the product of one minus the hazard at all previous times.

The hazard at $k$, $\Pr[D_k=1|D_{k-1}=0]$ can be estimated nonparametrically by dividing the number of cases during the interval $k$ by the number of individuals alive at the end of interval $k-1$.

This method is referred to as the Kaplan-Meier estimator, which is an excellent estimator of the survival curve, provided the total number of failtures over the follow up period is reasonably large.

In many cases, the number of cases during each interval is low (or even zero) and thus the nonparametric estimates of the hazard $\Pr[D_k=1|D_{k-1}=0]$ at $k$ can be very unstable. In this case, if our interest in estimation is of the hazard at a particular $k$, smoothing via a parametric model may be required.

## 17.3 Why censoring matters

It's also possible that individuals may start the follow-up at different dates (i.e., staggered entry into the study). In this setting it is helpful to define a time-varying indicator $C_k$ for censoring by time $k$. For each person at month $k$, the indicator $C_k$ takes value 0 if the administrative end of follow-up is greater than $k$ and 1 otherwise. In the person-time data format the row for a particular individual at time $k$ would have the indicator $C_{k+1}$.

The goal is to estimate the survival curve that *would have* been observed if nobody had been censored before $k_{end}$, where $k_{end}$ is the maximum administrative censoring time in the study.

In other words, our goal is to estimate the survival $\Pr[D_k=0|A=a]$ that would have been observed if the value of the time-varying indicators $D_k$ were known even after censoring. This can also be expressed as
$$\Pr[D_k^{\bar c=\bar 0}|A=a]$$

where $\bar c = (c_1,c_2,...,c_{k_{end}})$

The correct estimation of the survivals $\Pr[D_k=0|A=a]$ becomes the following. Assuming randomly assigned censoring (lol), the survival at $k$ is:
$$\Pr[D_k=0|A=a]=\prod_{m=1}^{k}\Pr[D_m=0|D_{m-1}=0,C_m=0,A=a]\text{ for }k<k_{end}$$

Like before we can just a nonparametric estimate or fit a logistic model.

## 17.4 IP weighting of marginal structural models

When the treated and the untreated are not exchangeable, a direct contrast of their survival curves cannot be endowed with a causal interpretation.

Define $D_k^{a,\bar c=\bar 0}$ as the counterfactual time-varying indicator for death at $k$ under treatment level $a$ and no censoring. We will shorten this to $D_k^a$ just for simplicity (LaTeX is time-consuming!). Furthermore, all expressions will assume $C_k=0$ but will not be included.

Supose we want to compare the counterfactual survivals $\Pr[D_{k+1}^{a=1}=0]$ and $\Pr[D_{k+1}^{a=0}=0]$. The causal question of interest is the comparison of the two for $k=0,2,...,k_{end}-1$.

Due to confounding, we cannot equate this to estimating $\Pr[D_{k+1}=0|A=1]$ and $\Pr[D_{k+1}=0|A=0]$.


Adjustment can be achieved through several methods, and this section is about using IP weighting. Estimationg of IP weighted survival curves has two steps.

First, we estimate the stabilized IP weights $SW^A$ for each individual in the study population.

Second, using the person-time data format, we fit a hazards model like below except that the individuals are weighted by their estimated SW^A. Technically, this IP weighted logistic model estimates the parameters of the marginal structural model
$$\text{logit}\Pr[D_{k+1}^a=0|D_k^a=0]=\beta_{0,k}+\beta_1a+\beta_2a\times k+\beta_3a\times k^2$$

In other words, the IP weighted model estimates the time-varying hazards that would have been observed if all individuals in the study population has been treated ($a=1$) and the time-varying hazards if they had been untreated ($a=0$).

Using methods from previous chapters, we can take the estimates of $\Pr[D_{k+1}^a=0|D_k^a=0]$ and math it into the survival $\Pr[D_{k+1}^a=0]$

## 17.5 The parametric g-formula

Another method to estimate the marginal survival curves (under the same assumptions as 17.4 which is exchangeability, positivity, and consistency) is standardization based on parametric models (i.e., the the parametric g-formula).

The survival $\Pr[D_{k+1}^a=0]$ at $k+1$ under treatment level $a$ is the weighted average of the survival conditional probabilities at $k+1$ within levels of the covariates $L$ and treatment $A=a$, with the proportion of individuals in each level $l$ of $L$ as the weights.

In other words,

$$\Pr[D_{k+1}^a=0]=\sum_l\Pr[D_{k+1}=0|L=l,A=a]\Pr[L=l]$$

Therefore, this means that the estimation of the parametric g-formula has two steps.

First, we need to estimate the conditional survivals $\Pr[D_{k+1}=0|L=l,A=a]$ using our administratively censored data.

Second, we compute the weighted average over all values $l$ of the covariates $L$.

## 17.6 G-estimation of structural nested models

The previous two sections describe causal contrasts that compare survivals or risks under different levels of treatment $A$. The survival was computed from hazards estimated by logistic regression models. This approach is feasible when the analytic method is IP weighting of marginal structural models or the parametric g-formual, but not when the method is g-estimation of structural nested models. Structural nested models are models for conditional causal contrasts (e.g., difference or ratio of covariate-specific means under different treatment levels), not for the components of those contrasts (e.g., each of the means under different treatment levels).

We can however consider a structural nested log-linear model to model the ratio of cumulative incidences under different treatment levels. *Structural nested cumulative failture time models* do precisely that, but they are best used when failure is a rare event.